<a href="https://colab.research.google.com/github/Berlingrad/MLinCyberSecurityProject/blob/BasicModelRun/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN Back Door Detector

## Preprocessing

###Data Download

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
!gdown https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
!gdown https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV

!mv /content/*.h5 /content/CSAW-HackML-2020/data

##!wget https://raw.githubusercontent.com/bolunwang/backdoor/master/visualizer.py

fatal: destination path 'CSAW-HackML-2020' already exists and is not an empty directory.
Downloading...
From: https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
To: /content/sunglasses_poisoned_data.h5
398MB [00:01, 225MB/s]
Downloading...
From: https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
To: /content/clean_validation_data.h5
716MB [00:02, 255MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
To: /content/clean_test_data.h5
398MB [00:09, 42.2MB/s]


In [2]:
import os
import gdown
import numpy as np
import random
import h5py
import glob2

import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")


In [3]:
def load_dataset(data_filename, keys=None):
    dataset = {}
    with h5py.File(data_filename, 'r') as hf:
        if keys is None:
            for name in hf:
                dataset[name] = np.array(hf.get(name))
        else:
            for name in keys:
                dataset[name] = np.array(hf.get(name))

    return dataset

def load_models(model_dir):
    models = []
    for net in glob2.glob("/content/CSAW-HackML-2020/models/*net.h5"):
        weights = glob2.glob(net.split("_")[0]+"*weights.h5")[0]
        print(net.split('/')[-1])
        m = load_model(net)
        m.load_weights(weights)

        models.append(m)
    return models

###Loading Data and Reshaping

In [4]:
# Loading Data
SG_poisoned_path = "/content/CSAW-HackML-2020/data/sunglasses_poisoned_data.h5"
SG_poisoned = load_dataset(SG_poisoned_path)
print(SG_poisoned.keys())
print(SG_poisoned['data'].shape)
print(SG_poisoned['label'].shape)

dict_keys(['data', 'label'])
(12830, 3, 55, 47)
(12830,)


In [5]:
clean_validation_path = "/content/CSAW-HackML-2020/data/clean_validation_data.h5"
valid_clean = load_dataset(clean_validation_path)
valid_clean.keys()
print(valid_clean['data'].shape)
print(valid_clean['label'].shape)

(11547, 3, 55, 47)
(11547,)


In [6]:
# Reshaping the data
SG_poisoned['data'] = SG_poisoned['data'].transpose((0,2,3,1))
valid_clean['data'] = valid_clean['data'].transpose((0,2,3,1))

###Data Parameters

In [7]:
##TODO

##SG_nClass = 
##vc_nClass = 

###Load Models

In [8]:
model_dir = "/content/CSAW-HackML-2020/models"

models = load_models(model_dir)

anonymous_bd_net.h5
sunglasses_bd_net.h5
multi_trigger_multi_target_bd_net.h5


In [9]:
# Using sunglasses_bd_net.h5 to predict the sunglasses_poisoned_data.h5
Y_pred_sunglasses_using_sunglasses_bn = np.argmax(models[1].predict(SG_poisoned['data']), axis = 1)
accuracy = np.mean(np.equal(Y_pred_sunglasses_using_sunglasses_bn, SG_poisoned['label']))*100
accuracy

99.94544037412315

In [17]:
Y_pred_val_sunglasses_using_sunglasses_bn = np.argmax(models[1].predict(valid_clean['data']), axis = 1)
accuracy = np.mean(np.equal(Y_pred_val_sunglasses_using_sunglasses_bn, valid_clean['label']))*100
accuracy

42.25339915129471

In [ ]:
# Repair the model

In [14]:
# Same thing using existing eval.py
!python3 CSAW-HackML-2020/eval.py CSAW-HackML-2020/data/sunglasses_poisoned_data.h5 CSAW-HackML-2020/models/sunglasses_bd_net.h5

2020-12-19 00:36:34.779653: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-19 00:36:41.897080: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-19 00:36:41.897970: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-19 00:36:41.902223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-19 00:36:41.902738: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-12-19 00:36:41.902774: I tensorflow/stream_executor/platform/default/dso_loade

##Reverse Engineering

In [16]:
##TODO
'''
for model in models:

    vs = Visualizer(model, )
'''

'\nfor model in models:\n\n    vs = Visualizer(model, )\n'